In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from net import *
from data import *
import numpy as np
import random
import math
import numpy as np
import crocoddyl
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
crocoddyl.switchToNumpyArray()

Using TensorFlow backend.


In [3]:
### Train, test data

data = getTrainingData(15000)

x_train , y_train = data[0:10000,0:3], data[0:10000, 3:]
print("TRAIN ",x_train.shape,",", y_train.shape)

x_test , y_test = data[10000:,0:3], data[10000:, 3:]
print("TEST ", x_test.shape,",", y_test.shape)

### Warmstart
x_valid = [] 
for _ in range(100):
    initial_config = [random.uniform(-1.99, 1.99), random.uniform(-1.99, 1.99), 0.0]
    x_valid.append(initial_config)
x_valid = np.array(x_valid)   

TRAIN  (10000, 3) , (10000, 153)
TEST  (5000, 3) , (5000, 153)


In [4]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()

x_train, x_test, x_valid = x_scaler.fit_transform(x_train), x_scaler.fit_transform(x_test), x_scaler.fit_transform(x_valid)
y_train, y_test = y_scaler.fit_transform(y_train), y_scaler.fit_transform(y_test)

In [5]:
net = kerasBaselineNet(x_train,
                     y_train,
                     x_test,
                     y_test,
                     NUNITS_INPUT = 64,
                     NUNITS = 128,
                     NHIDDEN = 2,
                     lr = 1e-3,
                     EPOCHS = 100,
                     BATCHSIZE = 64,
                     VERBOSE = 1                    
                    )

Epoch 1/100
10000/10000 [==============================] - 1s 67us/step - loss: 0.5028 - mean_squared_error: 0.5028 - mean_absolute_error: 0.5143
Epoch 2/100
10000/10000 [==============================] - 0s 38us/step - loss: 0.1976 - mean_squared_error: 0.1976 - mean_absolute_error: 0.2926
Epoch 3/100
10000/10000 [==============================] - 0s 40us/step - loss: 0.1019 - mean_squared_error: 0.1019 - mean_absolute_error: 0.1987
Epoch 4/100
10000/10000 [==============================] - 0s 41us/step - loss: 0.0835 - mean_squared_error: 0.0835 - mean_absolute_error: 0.1726
Epoch 5/100
10000/10000 [==============================] - 0s 42us/step - loss: 0.0676 - mean_squared_error: 0.0676 - mean_absolute_error: 0.1502
Epoch 6/100
10000/10000 [==============================] - 0s 41us/step - loss: 0.0601 - mean_squared_error: 0.0601 - mean_absolute_error: 0.1372
Epoch 7/100
10000/10000 [==============================] - 0s 37us/step - loss: 0.0504 - mean_squared_error: 0.0504 - mean_a

10000/10000 [==============================] - 0s 47us/step - loss: 0.0084 - mean_squared_error: 0.0084 - mean_absolute_error: 0.0333
Epoch 57/100
10000/10000 [==============================] - 0s 44us/step - loss: 0.0062 - mean_squared_error: 0.0062 - mean_absolute_error: 0.0318
Epoch 58/100
10000/10000 [==============================] - 0s 45us/step - loss: 0.0066 - mean_squared_error: 0.0066 - mean_absolute_error: 0.0304
Epoch 59/100
10000/10000 [==============================] - 0s 39us/step - loss: 0.0076 - mean_squared_error: 0.0076 - mean_absolute_error: 0.0331
Epoch 60/100
10000/10000 [==============================] - 0s 43us/step - loss: 0.0054 - mean_squared_error: 0.0054 - mean_absolute_error: 0.0289
Epoch 61/100
10000/10000 [==============================] - 0s 42us/step - loss: 0.0062 - mean_squared_error: 0.0062 - mean_absolute_error: 0.0305
Epoch 62/100
10000/10000 [==============================] - 0s 46us/step - loss: 0.0077 - mean_squared_error: 0.0077 - mean_absolut

In [7]:
y_predicted = net.predict_on_batch(x_valid)

In [8]:
x_valid = x_scaler.inverse_transform(x_valid)
y_predicted = y_scaler.inverse_transform(y_predicted)

In [11]:
# EXTRACT XS, US
xs_us = y_predicted[:,3:]

In [61]:
model = crocoddyl.ActionModelUnicycle()
model.costWeights = np.matrix([1, 0.3]).T
crocoddyl.switchToNumpyArray()
for initial, controls  in zip(x_valid, xs_us):
    state_ = controls.reshape(30,5)
    state = state_[:,0:3]
    #print(state.shape)
    control = state_[:,3:5]
    #print(control.shape)       
    
    xs = np.vstack((initial, state))
    
    ddp_xs = [np.matrix(i).T for i in xs]
    #print((ddp_xs))
    
    ddp_us = [np.matrix(i).T for i in control]
    #print(len(ddp_xs))

    assert len(ddp_xs) == len(ddp_us) + 1
    
    problem = crocoddyl.ShootingProblem(np.matrix(initial).T, [ model ] * 30, model)
    ddp = crocoddyl.SolverFDDP(problem)
    ddp.solve(ddp_xs, ddp_us, 1000)
    print(ddp.iter)
    

22
15
17
8
49
17
41
30
35
116
48
13
36
17
5
7
39
32
39
15
9
13
29
14
11
16
16
32
50
11
6
9
18
52
47
27
40
49
40
20
48
12
16
27
19
13
14
39
4
13
23
31
15
37
4
20
9
19
26
39
19
13
16
61
53
27
54
34
27
11
10
46
16
29
53
15
25
20
25
10
13
13
17
39
61
11
19
45
9
37
109
39
18
8
16
21
61
5
28
12


In [56]:
crocoddyl.switchToNumpyMatrix()

model = crocoddyl.ActionModelUnicycle()
model.costWeights = np.matrix([1, 0.3]).T

initial_config = [random.uniform(-1.99, 1.99), random.uniform(-1.99, 1.99), random.uniform(0.,1.)]
model.costWeights = np.matrix([1, 0.3]).T
problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 30, model)
ddp = crocoddyl.SolverDDP(problem)
ddp.solve([], [], 1000)


True

In [58]:
ddp.xs

[matrix([[ 0.72337179],
         [-1.35919298],
         [ 0.42601115]]), matrix([[ 1.04298943],
         [-1.21414987],
         [-0.11378261]]), matrix([[ 0.92429784],
         [-1.20058624],
         [-0.54923844]]), matrix([[ 0.61373232],
         [-1.01050217],
         [-0.76270888]]), matrix([[ 0.32913953],
         [-0.73853939],
         [-0.80568579]]), matrix([[ 0.09598839],
         [-0.49573084],
         [-0.73606731]]), matrix([[-0.08866819],
         [-0.32844974],
         [-0.60154959]]), matrix([[-0.2168114 ],
         [-0.24049043],
         [-0.4435461 ]]), matrix([[-0.28215193],
         [-0.20944585],
         [-0.29403515]]), matrix([[-0.29248656],
         [-0.2063164 ],
         [-0.17139762]]), matrix([[-0.26601415],
         [-0.21089867],
         [-0.08140422]]), matrix([[-0.22146595],
         [-0.21453311],
         [-0.02172953]]), matrix([[-0.17255753],
         [-0.21559603],
         [ 0.01361084]]), matrix([[-0.12733678],
         [-0.2149805 ],
   